# FundRaiser  
## Data Cleaning  

Overview: This notebook contains the code for the data cleaning and aggregation of:
* Census demographic data (2000 and 2010)
* IRS tax data (annual zipcode level values 2008-2014)
* Federal Election Commission data (2006-2014)  
    -- Individual donor data with donor name, address, amount and committee ID they donated to  
    -- Committee ID and Candidate ID linkages data  
    -- Candidate master data (all candidate names and party linked to unique FEC candidate ID)  

THL  
last updated July 13, 2019

In [1790]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

# Census data cleaning

In [1791]:
# pull in raw census data data and drop row 1, which are the descriptions of the contents, units etc., row 0 are the headers 
# 2000
sf12000 = pd.read_csv("data/census data/aff_download SF1 table/DEC_00_SF1_DP1_with_ann.csv",
                       skiprows=[1],  dtype={'GEO.id2': object})
# 2010
sf12010 = pd.read_csv("data/census data/aff_download SF1 table/DEC_10_SF1_SF1DP1_with_ann.csv",
                       skiprows=[1],  dtype={'GEO.id2': object})

In [1792]:
sf12010.dtypes.head(5)

GEO.id               object
GEO.id2              object
GEO.display-label    object
HD01_S001             int64
HD02_S001            object
dtype: object

The percentages (e.g., HC01_V01 in 2000, HD02_S001 in 2010) are being specified as objects. I will need to fix that but since there are 195 rows in this dataframe and I am not keeping all of them, I will do it at the end.

Now, read in the metadata for the census so I know what the column headers mean and which ones I want. Creating the column in the features20XXdf tables had to be done manually since the unique header names (e.g., HC01_V01) are different between the 2000 and 2010 census for the same data. For example, total population in the 2000 census data is HC01_VC01 but HD01_S001 in the 2010 census data.

In [1793]:
features2000 = pd.read_csv("data/census data/aff_download SF1 table/DEC_00_SF1_DP1_metadata_withchoosenfeatures.csv")
features2010 = pd.read_csv("data/census data/aff_download SF1 table/DEC_10_SF1_SF1DP1_metadata_withchosenfeatures.csv")

In [1794]:
# subset the census data to just the variables that we care about. 
namestomatch2000 = ['GEO.id', 'GEO.id2'] +  features2000.iloc[:,0].values.tolist()
namestomatch2010 = ['GEO.id', 'GEO.id2'] +  features2010.iloc[:,0].values.tolist()

In [1795]:
# subset 2000 and 2010 american family census data to just the features that I want.
amf2000 = sf1_2000.loc[ : , namestomatch2000]
amf2010 = sf1_2010.loc[ : , namestomatch2010]

In [1796]:
# create dictionaries to rename columns to more intuitive headers
newnames2000 = ['GEO.id', 'zipcode'] +  features2000.iloc[:,3].values.tolist()
newnames2010 = ['GEO.id', 'zipcode'] +  features2010.iloc[:,3].values.tolist()

dictionary2000 = dict(zip(namestomatch2000, newnames2000))
dictionary2010 = dict(zip(namestomatch2010, newnames2010))

amf2000.rename(columns = dictionary2000, inplace=True)
amf2010.rename(columns = dictionary2010, inplace=True)

In [1797]:
# bring in percent rural and percent urban from the other census data file. 
# Drop the 1st row that has descriptors and units, and just get the 5 columns I need.
urban2000 = pd.read_csv("data/census data/population urban vs rural/DEC_00_SF1_P002_with_ann.csv",
                        skiprows=[1], usecols = [0,1,3,4], dtype={'GEO.id2': object})
urban2010 = pd.read_csv("data/census data/population urban vs rural/DEC_10_SF1_P2_with_ann.csv",
                        skiprows=[1], usecols = [0,1,3,4], dtype={'GEO.id2': object})

I just want to know the percent of the total population that is urban vs. rural. Since one of those, urban or rural, gives me all the info I need I will just use percent urban.

In [1798]:
# rename the columns to make them more intuitive.
urban2000.columns = ['GEO.id', 'zipcode', 'total', 'urban']
urban2010.columns = ['GEO.id', 'zipcode', 'total', 'urban']

In [1799]:
# function to calculate percent of population to 1 decimal place. 
def percentpop(x,y):
    if y == 0:
        return 0.0
    else:
        return round(100*(x/y) ,1)

In [1800]:
# calculate the percent of total population that lives in an urban area. decimal place = 1 to match other percentages in the census data. 
urban2000['population_percenturban'] = np.vectorize(percentpop)(urban2000['urban'], urban2000['total'])
urban2010['population_percenturban'] = np.vectorize(percentpop)(urban2010['urban'], urban2010['total'])

In [1801]:
# subset urban2000 and urban2010 to just the identifiers and the percent urban column.
urban2000 = urban2000.loc[:, ['GEO.id', 'zipcode', 'population_percenturban']]
urban2010 = urban2010.loc[:, ['GEO.id', 'zipcode', 'population_percenturban']]

In [1802]:
# merge percent urban with the rest of the census data for 2000 and 2010.
census2000 = pd.merge(amf2000, urban2000, on=["GEO.id" , "zipcode"])
census2010 = pd.merge(amf2010, urban2010, on=["GEO.id" , "zipcode"])

In [1803]:
# drop GEO.id
census2000 = census2000.loc[:, 'zipcode': ]
census2010 = census2010.loc[:, 'zipcode': ]

In [1804]:
# specify all columns except zipcode as numeric.     
for col in  census2000.columns[1:]:
    census2000[col] = pd.to_numeric(census2000[col], errors='coerce')

In [1805]:
for col in  census2010.columns[1:]:
    census2010[col] = pd.to_numeric(census2010[col], errors='coerce')

There are a lot of additional zipcodes that I don't want to include (I have learned more about US zipcodes than I ever thought I could over the course of this project!). https://cber.cba.ua.edu/asdc/zip_zcta.html

I need to get rid of rows:
1. without zipcode info, or with zipcodes less than 5 digits.
2. with zip 00000 or 99999 (neither of which exist)
3. with 5 digits zips starting with 00xxx. There represent Puerto Rico, US territories and other locations outside the US like embassies.
4. with zip codes that have HH or XX as the last two digits. These are areas with little settlement; for example, parks, forest lands, and desert and mountainous areas where the USPS doesn't deliver mail.

In [1806]:
# x is a pandas dataframe with column 'zipcode'
def zipcodeclean(x):
    return x[(x['zipcode']!= "00000") 
                        & (x['zipcode']!= "99999") 
                        & ~(x.zipcode.str.len() < 5) 
                        & (x.zipcode.str.isnumeric() == True)
                        & (x.zipcode.str[0:2] != "00")]

In [1807]:
census2000 = zipcodeclean(census2000)
census2010 = zipcodeclean(census2010)

***
# IRS data cleaning

### 2008
The 2008 IRS data is formatted differently than the other years so I am going to clean it separately and then combine it. Specifically, the 'agi_stubs' columns need to be aggregated for every zipcode to match the format of the other years that do not have data broken out by agi_stub (which is adjusted gross income classes).

In [1808]:
irs2008 = pd.read_csv("data/income_by_zip/08zpall.csv", dtype={'state': object, 'ZIPCODE': object})
irs2008.columns = irs2008.columns.str.lower() # convert all column names to lowercase. 
irs2008 = irs2008.rename(columns = {'agi_class':'agi_stub'}) # rename this column to match the other irs years

In [1809]:
irs2008aggregated = irs2008.groupby(['state', 'zipcode'], as_index = False).sum()

In [1810]:
colswanted = ['state'
              , 'zipcode'
              , 'agi_stub'
              , 'n1' # number of returns; proxy for number of households
              , 'n2' # number of exemptions; proxy for population
              , 'numdep' # number of dependents
              , 'a00100' # Adjusted gross income (AGI)
              , 'a01700' # Pensions and annuities in AGI (might be indicative of older population)
              , 'a18425' # State and Local Income taxes paid
              , 'a18450' # State and local general sales taxes paid
              , 'a18500' # Real estate taxes paid
              , 'a18300' # Total taxes paid
              , 'af5695' # Residential energy tax credit; proxy for beliefs on environmental policy or causes?
              , 'a07220' # Child tax credit
              , 'a06500' # Income tax 
             ]

In [1811]:
irs2008fixed = irs2008aggregated.loc[ : , colswanted]

Change the dollar amounts for each zipcode so they are in 1000's of dollars and make this value be the average for each household in a given zipcode.

In [1812]:
def averageperhousehold(x, y):
    return round((x/1000)/y,5)

In [1813]:
for col in  irs2008fixed.columns[6:]:
    irs2008fixed[col] = np.vectorize(averageperhousehold)(irs2008fixed[col], irs2008fixed['n1'])

Also, turn the number of dependents into average number of dependents per household.

In [1814]:
irs2008fixed.numdep = round(irs2008fixed.numdep/irs2008fixed.n1, 2)

Add in a year column, and rename "af5695" to "a07260" to match the other years. 

In [1815]:
irs2008fixed = irs2008fixed.rename(columns = {'af5695':'a07260'})
irs2008fixed['year'] = 2008

### 2011 
Needs to have leading zeros added to the eastern state zipcodes.

In [1816]:
irs2011 = pd.read_csv('data/income_by_zip/11zpallagi.csv', dtype={'STATE': object, 'zipcode': object})

In [1817]:
# add a leading zero to all zipcodes and then substring to the last 5 characters
#irs2011.zipcode = '0' + irs2011.zipcode.astype(str)
irs2011.zipcode = irs2011.zipcode.apply(lambda x: x[len(x)-5:len(x)])

Now that this is fixed write it as a csv so I don't have to worry about it down the road again. 

In [1818]:
export_csv =  irs2011.to_csv ('data/income_by_zip/11zpallagi.csv', index = None, header=True)

### 2009 - 2014

In [1819]:
import glob
fname = glob.glob("data/income_by_zip/*pallagi.csv")

In [1820]:
fname

['data/income_by_zip/12zpallagi.csv',
 'data/income_by_zip/11zpallagi.csv',
 'data/income_by_zip/14zpallagi.csv',
 'data/income_by_zip/13zpallagi.csv',
 'data/income_by_zip/09zpallagi.csv',
 'data/income_by_zip/10zpallagi.csv']

In [1821]:
def irsclean(x):
    colswanted = [ 'year'
                  ,'state'
                  , 'zipcode'
                  , 'agi_stub'
                  , 'n1' # number of returns; proxy for number of households
                  , 'n2' # number of exemptions; proxy for population
                  , 'numdep' # number of dependents
                  , 'a00100' # Adjusted gross income (AGI)
                  , 'a01700' # Pensions and annuities in AGI (might be indicative of older population)
                  , 'a18425' # State and Local Income taxes paid
                  , 'a18450' # State and local general sales taxes paid
                  , 'a18500' # Real estate taxes paid
                  , 'a18300' # Total taxes paid
                  , 'a07260' # Residential energy tax credit; proxy for beliefs on environmental policy or causes?
                  , 'a07220' # Child tax credit
                  , 'a06500' # Income tax 
                 ]
    tmp = pd.read_csv(x, dtype={'STATE': object, 'zipcode': object})
    tmp['year'] = '20'+ x[19:21] # get the 4 digit year for each irs csv file from the file name.
    tmp.columns = tmp.columns.str.lower() # make all column names lower case
    tmp = tmp.loc[ : , colswanted]
      
    return tmp
    

In [1822]:
irs09to14 = []
for i in range(0, len(fname)-1):
    data = irsclean(fname[i])
    # store DataFrame in list
    irs09to14.append(data)
irs09to14 = pd.concat(irs09to14, axis=0) # concat to a single df

In [1823]:
# fix the column data types to they are all numbers
#for col in irs09to14.columns[6:]:
#    irs09to14[col] = np.vectorize(averageperhousehold)(irs09to14[col], irs09to14['n1'])    

Fix the number of dependents to be average per household in each zip. This is done separately from the other columns above because I am using a different number of significant digits to match the census data.

In [1824]:
irs09to14.numdep = round(irs09to14.numdep/irs09to14.n1, 2)  

In [1825]:
# put the 2008 columns in the same order as irs09to14
yy = list(irs09to14.columns)
irs2008fixed = irs2008fixed[yy]

In [1826]:
irs = pd.concat([irs2008fixed,irs09to14], axis = 0, ignore_index = True)

In [1827]:
irs = zipcodeclean(irs)

Aggregate irs for each zipcode to election cycle by averaging the values for each 2 year period (e.g. 2010 election cycle = 2009 and 2010 data).

In [1828]:
# add in fecyear
#irs['fecyear'] = 2 * np.ceil(irs.year.astype('int')/2)
#irs['fecyear'] = irs.fecyear.astype('object')

In [1829]:
# x here is a year column from a pd.df
def makefecyear(x):
    tmp =  round(2 * np.ceil(x.astype(str).astype(int)/2))
    return tmp.astype('int')

In [1830]:
irs['fecyear'] = makefecyear(irs.year)

In [1831]:
# drop year
irs = irs.drop(['year'], axis = 1)
# aggregate by fecyear
irsbyfeccyle = irs.groupby(['fecyear', 'state', 'zipcode'], as_index = False).mean()

### Merge census and irs data
The 2000 census data gets merged with irs.fecyear = 2008, and all other irs feccycles are merged with 2010 census data.

In [1832]:
irsfec2008 = irsbyfeccyle[(irsbyfeccyle['fecyear'] == 2008)]
censusirs2008 =  pd.merge(census2000, irsfec2008, on = 'zipcode', how = 'outer')

In [1833]:
irsfec2010on = irsbyfeccyle[(irsbyfeccyle['fecyear'] != 2008)]
censusirs2010on =  pd.merge(census2010, irsfec2010on, on = 'zipcode', how = 'outer')

In [1834]:
# reorder columns to match
yy = list(censusirs2010on.columns)
censusirs2008 = censusirs2008[yy]

In [1835]:
census_irs = pd.concat([censusirs2008,censusirs2010on], axis = 0, ignore_index = True)

***

# FEC data cleaning

This is going to use SQLite to pull in and doing some of the data pre-processing. The FEC donor database from 2007-2014 for the entire US was approximately 11 million rows. 

In [1836]:
conn = sqlite3.connect('data/database_Jun9.db') 

In [1837]:
uquery = """SELECT SUBSTR(m.ZIP_CODE, 1,5) as zipcode, 
CAND_PTY_AFFILIATION as party, SUBSTR(TRANSACTION_DT,-4) as year, 
SUM(TRANSACTION_AMT) as amt, SUM(1) as ndon 
FROM (SELECT CMTE_ID, ZIP_CODE, TRANSACTION_DT,
(CASE
    WHEN TRANSACTION_AMT > 5400 THEN 5400 
    ELSE TRANSACTION_AMT END) TRANSACTION_AMT 
    FROM contribs 
    WHERE TRANSACTION_AMT >= 200
    ) m 
JOIN (SELECT DISTINCT CAND_ID, CMTE_ID FROM links) l ON m.CMTE_ID == l.CMTE_ID 
JOIN (SELECT DISTINCT CAND_ID, CAND_NAME, 
CASE upper(CAND_PTY_AFFILIATION) WHEN 'DEM' THEN 'DEM' WHEN 'REP' THEN 'REP' ELSE 'THR' END CAND_PTY_AFFILIATION
    FROM cand_4) c ON c.CAND_ID = l.CAND_ID 
GROUP BY 1, 2, 3;"""

In [1838]:
contribs = pd.read_sql_query(uquery, conn)
conn.close()

In [1839]:
# drop columns where year is blank or year < 2004 or year > 2015, year is an object in this df
lis = range(2005,2014)
yearswanted = ["{:02d}".format(x) for x in lis]
contribs = contribs[(contribs['year'].isin(yearswanted))]

In [1840]:
# drop row without zipcodes,from Puerto Rico, and outside US (territories, embassies etc.) start with 00.
contribs = zipcodeclean(contribs)

In [1841]:
# add in FEC year
contribs['fecyear'] = makefecyear(contribs.year) 

In [1842]:
# drop year
contribs = contribs.drop(['year'], axis = 1)

In [1843]:
contribsbyFEC = contribs.groupby(['fecyear', 'zipcode', 'party'], as_index = False).sum()

For every zipcode also get the amount and ndon from last election cycle. Basically take the FEC data and offset if by one cycle then remerge.

In [1844]:
# create a copy of contribsbyFEC and rename columns 
feclast = contribsbyFEC.copy()
feclast = feclast.rename(columns = {'fecyear': 'lastfecyear', 'amt': 'amtlastcycle', 'ndon': 'ndonlastcycle' })

In [1845]:
# remake the fecyear column for the merge.
feclast['fecyear'] = feclast.lastfecyear.astype(int) + 2

In [1846]:
# merge with the contribsbyFEC by fecyear to get the offsets correct.
fecwithlast =  pd.merge(contribsbyFEC, feclast, on =['fecyear', 'zipcode', 'party'], how = 'outer')

In [1847]:
# get rid of the lastfecyear column as it was only needed for the merge.
fecwithlast = fecwithlast.drop(['lastfecyear'], axis = 1)

In [1848]:
# drop the 2006 election cycle as is was only included to create these last cycle amt and ndon colums, fecyear 2016
fecwithlast = fecwithlast[(fecwithlast['fecyear'] != 2006) & (fecwithlast['fecyear'] != 2016)]

In [1849]:
# if amt or ndon last cycle is an NaN then set to zero.
values = {'amtlastcycle': 0, 'ndonlastcycle': 0}
fec = fecwithlast.fillna(value = values)

The rest of the data cleaning was completed in R to take advantage of some geospatial packages for imputing missing values of target features based on averages of proximal zipcodes. 